# Defining Jobs using Firetasks

This tutorial shows you how to:

* Run multiple tasks within a single Firework

* Run tasks that are defined within a Python function, rather than a shell script

## Introduction to Firetasks

In the Introductory tutorial, we ran a simple script that performed ```echo "howdy, your job launched successfully!" >> howdy.txt"```. Looking inside fw_test.yaml, recall that the command was defined within a task labeled ScriptTask:

```spec:
  _tasks:
  - _fw_name: ScriptTask
    script: echo "howdy, your job launched successfully!" >> howdy.txt
```

The ScriptTask is one type of Firetask, which is a predefined job template written in Python. The ScriptTask in particular refers Python code inside FireWorks that runs an arbitrary shell script that you give it. You can use the ScriptTask to run almost any job (without worrying that it’s all done within a Python layer). However, you might want to set up jobs that are more powerful than shell scripts using Python programming. Later in this section, we’ll demonstrate how to accomplish this with custom Firetasks. However, first we’ll demonstrate the simplest version to linearly run multiple tasks.

## Running multiple Firetasks

You can run multiple tasks within the same Firework. For example, the first step of your Firework might write an input file that the second step reads and processes. Finally, a third step might move the entire output directory somewhere else on your filesystem (or a remote server, or insert results in a database).

Let’s create a Firework that:

* Writes an input file based on a template with some substitutions applied. We’ll do this using a built-in TemplateWriterTask that can help create such files.

* Executes a script using ScriptTask that reads the input file and produces some output. In our test case, it will just count the number of words in that file. However, this code could be any program, for example a chemistry code.

* Copies all your outputs to your home directory using FileTransferTask.

The three-step Firework thus looks like this:

![firetask](images/templatetask.png)

Let's create our three-step Firework with python:

In [ ]:
from fireworks import Firework, FWorker, LaunchPad, ScriptTask, TemplateWriterTask, FileTransferTask
from fireworks.core.rocket_launcher import launch_rocket

# set up the LaunchPad and reset it
launchpad = LaunchPad.auto_load()
launchpad.reset('', require_password=False)

# create the Firework consisting of multiple tasks
firetask1 = TemplateWriterTask({'context': {'opt1': 5.0, 'opt2': 'fast method'}, 'template_file': 'simple_template.txt', 'output_file': 'inputs.txt'})
firetask2 = ScriptTask.from_str('wc -w < inputs.txt > words.txt')
firetask3 = FileTransferTask({'files': [{'src': 'words.txt', 'dest': '~/words.txt'}], 'mode': 'copy'})
fw = Firework([firetask1, firetask2, firetask3])

# store workflow and launch it locally, single shot
launchpad.add_wf(fw)

Let's play around in the terminal with ```lpad``` to look at what is in the database and then submit our job using ```qlaunch```.

After having run this firework, you should see two files written out to the system, inputs.txt and words.txt, confirming that you successfully ran the first two steps of your job! You can also navigate to your home directory and look for words.txt to make sure the third step also got completed correctly.

## Creating a custom Firetask

The TemplateWriterTask, ScriptTask, FileTransferTask are built-into FireWorks and can be used to perform useful operations. In fact, they might be all you need! In particular, because the ScriptTask can run arbitrary shell scripts, it can in theory run any type of computation and is an ‘all-encompassing’ Firetask. ScriptTask also has many additional features (see Fireworks documentation).

However, if you are comfortable with some basic Python, you can define your own custom Firetasks for the codes you run. A custom Firetask gives you more control over your jobs, clarifies the usage of your code, and guards against unintended behavior by restricting the commands that can be executed.

Here, we’ll be creating a custom Firetask that adds one or more numbers using Python’s sum() function, and later building workflows using this (and similar) Firetasks.

### How FireWorks bootstraps a job

Before diving into an example of custom Firetask, it is worth understanding how FireWorks is bootstrapping jobs based on your specification. The basic process looks like this:

![firetask](images/spec_sketch.png)


1. The first step of the image just shows how the spec section of the Firework is structured. There is a section that contains your Firetasks (one or many), as we saw in the previous examples. The spec also allows you to define arbitrary JSON data (labeled input in the diagram) to pass into your Firetasks as input. So far, we haven’t seen an example of this; the only information we gave in the spec in the previous examples was within the _tasks section.

2. In the second step, FireWorks dynamically loads Python objects based on your specified _tasks. It does this by searching a list of Python packages for Python objects that have a value of _fw_name that match your setting. When we set a _fw_name of ScriptTask in the previous examples, FireWorks was loading a Python object with a _fw_name class variable set to ScriptTask (and passing the script parameter to its constructor). The ScriptTask is just one type of Firetask that’s built into FireWorks to help you run scripts easily. You can write code for custom Firetasks anywhere in the user_packages directory of FireWorks, and it will automatically be discovered. If you want to place your Firetasks in a package outside of FireWorks, please read the FireWorks configuration tutorial. You will just need to define what Python packages to search for your custom Firetasks, or use a special format that allows for direct loading of classes.

3. In the third step, we execute the code of the Firetask we loaded. Specifically, we execute the run_task method which must be implemented for every Firetask. FireWorks passes in the entire spec to the run_task method; the run_task method can therefore modify its behavior based on any input data present in the spec, or by detecting previous or future tasks in the spec.

4. When the Firetask is done executing, it returns a FWAction object that can modify the workflow (or continue as usual) and pass information to downstream FireWorks.

### Custom Firetask example: Addition Task

Let’s explore custom Firetasks with an example: a custom Python script for adding two numbers specified in the spec.

Let’s first look at what a custom Firetask looks like in Python. Look inside the file addition_task.py which defines the Addition Task:

```from fireworks import FiretaskBase, FWAction

class AdditionTask(FiretaskBase):

   _fw_name = "Addition Task"

   def run_task(self, fw_spec):
       input_array = fw_spec['input_array']
       m_sum = sum(input_array)

       print("The sum of {} is: {}".format(input_array, m_sum))

       return FWAction(stored_data={'sum': m_sum}, mod_spec=[{'_push': {'input_array': m_sum}}])
```


A few notes about what’s going on (things will be clearer after the next step):

* In the class definition, we are extending FiretaskBase to tell FireWorks that this is a Firetask.

* A special parameter named _fw_name is set to Addition Task. This parameter sets what this Firetask will be called by the outside world and is used to bootstrap the object, as described in the previous section.

* The run_task() method is a special method name that gets called when the task is run. It can take in a Firework specification (spec) in order to modify its behavior.

* When executing run_task(), the AdditionTask we defined first reads the input_array parameter of the Firework’s spec. It then sums all the values it finds in the input_array parameter of the Firework’s spec using Python’s sum() function. Next, the Firetask prints the inputs and the sum to the standard out. Finally, the task returns a FWAction object.

* The FWAction is giving two instructions. The first says we should store the sum we computed in the database (inside the Firework’s stored_data section). The second will pass the results on to any downstream FireTask or FireWork in the workflow as part of the spec inside a key called input_array.



Now let’s define a Firework that runs this Firetask to add the numbers 1 and 2. Look inside the file fw_adder.yaml for this new Firework definition:

```
spec:
  _tasks:
  - _fw_name: Addition Task
    parameters: {}
  input_array:
  - 1
  - 2
```

Let's add this to the Fireworks database:

In [ ]:
from fireworks import Firework, FWorker, LaunchPad
from fireworks.core.rocket_launcher import launch_rocket
from fw_tutorials.firetask.addition_task import AdditionTask

# set up the LaunchPad and reset it
launchpad = LaunchPad.auto_load()
launchpad.reset('', require_password=False)

# create the Firework consisting of a custom "Addition" task
firework = Firework(AdditionTask(), spec={"input_array": [1, 2]})

# store workflow and launch it locally
launchpad.add_wf(firework)

... and submit our job to the queue:

In [ ]:
!qlaunch singleshot

Confirm that the sum is not only printed to the screen, but also stored in our Firework in the stored_data section:

In [ ]:
!lpad get_fws -i 1 -d all